In [1]:
import os
import json
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random
import sys
sys.path.append('../src/dataset')
from mapillary_dataset import Mapillary_SemSeg_Dataset, N_LABELS_v1_2
import albumentations as A
sys.path.append('../src')
from models import UNet
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F

# sys.path.append('../src/models/deeplabv3_model')
from models.deeplabv3_model import DeepLabV3

/home/uzohchinedu/.local/lib/python3.8/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.19 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
! ls ../data

LICENSE  config_v1.2.json  demo.py  training
README	 config_v2.0.json  testing  validation


In [3]:
train_dataset = Mapillary_SemSeg_Dataset('training', '../data', (512, 512), augment=False, verbose=False)
test_dataset = Mapillary_SemSeg_Dataset('testing', '../data', (512, 512), augment=False, verbose=False)

In [4]:
loader_args = dict(batch_size=2, num_workers=os.cpu_count()//2, pin_memory=True)
train_loader = DataLoader(train_dataset, shuffle=True, **loader_args)
test_loader = DataLoader(test_dataset, shuffle=True, **loader_args)

In [5]:
train_loader = iter(train_loader)
test_loader = iter(test_loader)

In [6]:
# checkpoint_dir = '2ff7a424-9b16-44f6-956d-7c9d5b06c6d2'
# epoch=29

In [7]:
model = DeepLabV3(n_channels=3, n_classes=N_LABELS_v1_2, model_id = 1, project_dir="..")
# model = UNet(n_channels=3, n_classes=N_LABELS_v1_2, bilinear=False)
model = model.to(memory_format=torch.channels_last)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

pretrained resnet, 18


In [8]:
images, masks_true = next(train_loader)

In [9]:
images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)


In [10]:
masks_pred = model(images)

/home/uzohchinedu/mapillary/notebooks/../src/models/deeplabv3_model/aspp.py:46: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  out_img = F.upsample(out_img, size=(feature_map_h, feature_map_w), mode="bilinear") # (shape: (batch_size, 256, h/16, w/16))
/home/uzohchinedu/mapillary/notebooks/../src/models/deeplabv3_model/deeplabv3.py:38: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  output = F.upsample(output, size=(h, w), mode="bilinear") # (shape: (batch_size, num_classes, h, w))


In [11]:
masks_pred = masks_pred.argmax(dim=1)

In [12]:
images[0].shape, masks_pred[0].shape

(torch.Size([3, 512, 512]), torch.Size([512, 512]))